In [24]:
import pandas
import os
import re
import glob
import shutil
import unidecode
from IPython.display import HTML

In [57]:
df = pandas.read_csv('Papers.csv', delimiter='\t')
Submission_Folder = 'CameraReadys 4-34'
Destinatio_Folder = 'accepted_papers'
os.makedirs(Destinatio_Folder, exist_ok=True)

In [65]:
def get_submission_filename(paper_id):
    path = os.path.join(Submission_Folder, str(paper_id), 'CameraReady')
    files = glob.glob(os.path.join(path, '*.pdf'))
    assert len(files) <= 1
    if len(files) == 0:
        return ''
    else:
        return files[0]

get_submission_filename(3)

''

In [35]:
HTML(df.to_html())

,Paper ID,Created,Last Modified,Paper Title,Abstract,Primary Contact Author Name,Primary Contact Author Email,Authors,Author Names,Author Emails,Primary Subject Area,Secondary Subject Areas,Conflicts,Assigned,Completed,% Completed,Bids,Discussion,Status,Requested For Author Feedback,Author Feedback Submitted?,Requested For Camera Ready,Camera Ready Submitted?,Requested For Presentation,Files,Number of Files,Supplementary Files,Number of Supplementary Files,Reviewers,Reviewer Emails,MetaReviewers,MetaReviewer Emails,SeniorMetaReviewers,SeniorMetaReviewerEmails
0,3,2/11/2022 12:26:35 AM -08:00,3/27/2022 8:47:13 PM -07:00,Improving Forecasting Demand for Maintenance Spare Parts,"The need to reduce inventory holding costs and increasing system operational availability are the main motivation behind improving spare parts inventory management in a major power utility company in Medina. This paper reports in an effort made to optimize the orders quantities of spare parts by improving the method of forecasting the demand. The study focuses on equipment that has frequent spare part’s purchase orders with uncertain demand. The pattern of the demand considers a lumpy pattern which makes conventional forecasting methods less effective. A comparison was made by benchmarking various methods of forecasting based on experts’ criteria to select the most suitable method for the case-study. Three actual data sets were used to make the forecast in this case study. Two neural networks (NN) approaches were utilized and compared, namely long short-term memory (LSTM) and multilayer perceptron (MLP). The results as expected, showed that the NN models gave better results than traditional forecasting method (judgmental method). In addition, the LSTM model had a higher predictive accuracy than the MLP model.",ABDULAZIZ AFANDI,Afndi4@hotmail.com,ABDULAZIZ AFANDI (Islamic University of Medina)*,"AFANDI, ABDULAZIZ*",Afndi4@hotmail.com*,NaN,NaN,0,2,2,100,0,Disabled (0),Desk Reject,No,No,No,No,No,"Improving Forecasting Demand for Maintenance Spare Parts v7.pdf (773,351 bytes)",1,NaN,0,Adam Kortylewski (Max Planck Institute for Informatics); Swetasudha Panda (Oracle Labs),akortyle@mpi-inf.mpg.de; swetasudha.panda@oracle.com,NaN,NaN,NaN,NaN
1,4,2/19/2022 1:37:31 AM -08:00,4/9/2022 10:55:01 AM -07:00,Lost In Translation: Generating Adversarial Examples Robust to Round-Trip Translation,"Language Models today provide a high accuracy across a large number of downstream tasks. However, they remain susceptible to adversarial attacks, particularly against those where the adversarial examples maintain considerable similarity to the original text. Given the multilingual nature of text, the effectiveness of adversarial examples across translations and how machine translations can improve the robustness of adversarial examples remain largely unexplored. In this paper, we present a comprehensive study on the robustness of current text adversarial attacks to round-trip translation. We demonstrate that 6 state-of-the-art text-based adversarial attacks do not maintain their efficacy after round-trip translation. Furthermore, we introduce an intervention-based solution to this problem, by integrating Machine Translation into the process of adversarial example generation and demonstrating an increased robustness to round-trip translation. Our results indicate that finding adversarial examples robust to round-trip translation can help identify insufficiency of language models that is common across languages, and motivate further research into multilingual adversarial attacks.",Neel Bhandari,neelbhandari64@gmail.com,Neel Bhandari (RV College of Engineering)*; Pin-Yu Chen (IBM Research AI),"Bhandari, Neel*; Chen, Pin-Yu",neelbhandari64@gmail.com*; pinyuchen.tw@gmail.com,NaN,NaN,1,2,2,100,0,Disabled (0),Accept,No,No,Yes,Yes,No,"Lost_In_Translation_SRML.pdf (265,129 bytes)",1,NaN,0,"Rajkumar Theagarajan (University of California, Riverside); Xinchen Yan (Waymo)",rthea001@ucr.edu; xc

In [66]:
accepted = open('accepted.html', 'w')

for i, p in df.loc[df['Status'] == 'Accept'][['Paper ID', 'Authors', 'Paper Title']].iterrows():
    paper_id = int(p[0])
    show_title = p[2].capitalize()
    show_name = p[1].replace('*', '')
    # Get first author's last name with accent removed.
    first_author = show_name.strip().split(';')[0]
    last_name = first_author[:first_author.find('(')].strip().split(' ')[-1]
    last_name = unidecode.unidecode(last_name)
    # Get the first word in title
    cleaned_title = re.sub("[^0-9a-zA-Z ]+", "", p[2])
    short_title = cleaned_title.split()[0]
    if short_title.lower() in ['a', 'the', 'an']:
        short_title = cleaned_title.split()[1]
    # Move file to a new folder.
    old_filename = get_submission_filename(paper_id)
    new_filename = f'{last_name}22{short_title}.pdf'.lower()
    # print(paper_id, old_filename, new_filename)
    if old_filename:
        shutil.copy(old_filename, os.path.join(Destinatio_Folder, new_filename))
        url = f'https://download.huan-zhang.com/events/srml2022/accepted/{new_filename}'
    else:
        print(f'paper not found for {paper_id}')
        url= '#'
    # Print HTML.
    accepted.write(f'''<li> <a href="{url}">{show_title}</a>
      <p>{show_name}</p>
    </li>\n''')
    accepted.flush()
accepted.close()


paper not found for 32
paper not found for 33


In [52]:
reviewers = open('reviewers.html', 'w')
r_list = set()
for r in df['Reviewers']:
    if isinstance(r, str):
        for rr in r.split(';'):
            r_list.add(rr.strip())

for r in r_list:
    reviewers.write(f'<li>{r}</li>\n')
reviewers.close()